### Read data from influxdb

In [ ]:
from data_exploration import *

In [ ]:
client, client_df = connect_to_influxdb()

In [ ]:
df = get_all_data(client_df)

### Add additional columns for grouping

In [ ]:
df["year"]=df["time"].dt.year
df["month"]=df["time"].dt.month
df["day"]=df["time"].dt.day
df["hour"]=df["time"].dt.hour
df["weekday"]=df["time"].dt.weekday_name
df["weekday"] = pd.Categorical(df["weekday"], ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"])

In [ ]:
df.head()

## Aggregated data by device

In [ ]:
## helper function
def show_data(ev):
    clear_output(wait=True)
    display(Box(children = [device_name1,measurement_type1,graph_type1,aggregated_by1,show_button1]))
                
    subset_iperf = df[(df["test_type"]=="iperf") & (df["SK_PI"] == device_name1.value) & (df["MES_TYPE"] == measurement_type1.value)]
    subset_speedtest= df[(df["test_type"]=="speedtest") & (df["SK_PI"] == device_name1.value)& (df["MES_TYPE"] == measurement_type1.value)]
    stat_iperf = subset_iperf[[aggregated_by1.value,"result"]].groupby(aggregated_by1.value).agg(["mean","max","size"])["result"]
    stat_speedtest = subset_speedtest[[aggregated_by1.value,"result"]].groupby(aggregated_by1.value).agg(["mean","max","size"])["result"]
        
    fig = get_fig_agg_by_device(subset_iperf,subset_speedtest,stat_iperf,stat_speedtest,
                           aggregated_by1.value,graph_type1.value, measurement_type1.value,
                           df[aggregated_by1.value].sort_values().unique(),"Data by device: ")
    fig.show()

In [ ]:
device_name1 = widgets.Dropdown(options = df['SK_PI'].sort_values().unique(), description ='Device number: ',style = {'description_width': 'initial'}, disabled=False)
measurement_type1 =  widgets.Dropdown(options = df['MES_TYPE'].unique(), description ='Measurement type: ',style = {'description_width': 'initial'}, disabled=False)
graph_type1 = widgets.Dropdown(options = ["boxplot","mean",'max'], description ='Graph type: ',style = {'description_width': 'initial'}, disabled=False)
aggregated_by1 = widgets.Dropdown(options = ["year","month","day","weekday","hour"], description ='Aggregated by: ',style = {'description_width': 'initial'}, disabled=False)

show_button1 = widgets.Button(button_style= 'info', description="Show Data")
show_button1.on_click(show_data)

display(Box(children = [device_name1,measurement_type1,graph_type1,aggregated_by1,show_button1]))

## Aggregated data for all devices over the last 6 months

In [ ]:
## helper function
def show_data_all(ev):
    clear_output(wait=True)
    display(Box(children = [test_type2,measurement_type2,graph_type2,aggregated_by2,show_button2]))
    
    subset = df[(df["test_type"]==test_type2.value) &  (df["MES_TYPE"] == measurement_type2.value) &(df["time"]>  datetime.now() - pd.DateOffset(months=6))]
    subset=subset.sort_values(aggregated_by2.value)
    
    stat = subset[[aggregated_by2.value,"result"]].groupby(aggregated_by2.value).agg(["mean","max","size"])["result"]
    fig = get_fig_agg_all(subset,stat,test_type2.value,
                           aggregated_by2.value,graph_type2.value,
                           subset[aggregated_by2.value].sort_values().unique(),"Summary for all devices over the last 6 months: (iperf in red, speedtest in green)")
    fig.show()

In [ ]:
test_type2 =  widgets.Dropdown(options = df['test_type'].unique(), description ='Test type: ',style = {'description_width': 'initial'}, disabled=False)
measurement_type2 =  widgets.Dropdown(options = df['MES_TYPE'].unique(), description ='Measurement type: ',style = {'description_width': 'initial'}, disabled=False)
graph_type2 = widgets.Dropdown(options = ["boxplot","mean",'max'], description ='Graph type: ',style = {'description_width': 'initial'}, disabled=False)
aggregated_by2 = widgets.Dropdown(options = ["year","month","day","weekday","hour"], description ='Aggregated by: ',style = {'description_width': 'initial'}, disabled=False)

show_button2 = widgets.Button(button_style= 'info', description="Show Data")
show_button2.on_click(show_data_all)

display(Box(children = [test_type2,measurement_type2,graph_type2,aggregated_by2,show_button2]))